### clean lexicon (加上##的規則)

In [1]:
# 2021/12   : 關閉 OutPutLexicon.merge_non_english() 改使用 sub('[^\u4e00-\u9fa5A-Za-z0-9]+','')
# 2021/12/24: 字典中只留下中文、英文、數字

In [1]:
import requests
import json
import re

counter = 0

class OutPutLexicon():
    '''
    Output lexicon txt file of each class in cilin 
    call "start()" function to get lexicon txt file
    '''
    def __init__(self,file_name):

        #with open(file_name, newline='', encoding="utf-8") as jsonfile: # 本地json
        #    self.data = json.load(jsonfile)
        
        #self.data = requests.get(url='http://127.0.0.1:5000/getCilinJson').json()
        self.data = requests.get(url='http://140.118.170.215:7000/getCilinJson').json() # get cilin json (實驗室)
        #self.data = requests.get(url='http://140.118.170.215:5000/getCilinJson').json() # get cilin json (實驗室)
        #self.data = requests.get(url='http://122.116.133.54:5000/getCilinJson').json() # 可東ip
        
    def start(self):
        """
        Output lexicon txt file of each class in cilin 
        """
        counter = 0   # count total number of word in lexicon
        self.export_class_name = []
        all_class_name = self.data.keys()
        
        for class_name in all_class_name:
            class_lexicon = self.class_lexicon(class_name)  # get lexicon of corresponding class name
            print(class_name,"/",len(class_lexicon))
            
            if class_name == 'Brand 品牌': # since we have both cilin lexicon and pchome lexicon, we need to merge them together
                pchome_brand = self.read_pchome_file(r'C:\Users\EE303\OneDrive\PChome\相似同義詞輸出\pchome_data\brand.txt')
                merge_lexicon = class_lexicon.union(pchome_brand) # merge cilin lexicon and pchome lexicon together (union)
                self.write_txt(class_name,merge_lexicon)
                print("merge",class_name,"/",len(merge_lexicon))
                counter = counter + len(merge_lexicon)
                
            if class_name == 'Type 種類':
                pchome_type = self.read_pchome_file(r'C:\Users\EE303\OneDrive\PChome\相似同義詞輸出\Lexicon\Type_pchome_only.txt')
                merge_lexicon = class_lexicon.union(pchome_type) # merge cilin lexicon and pchome lexicon together (union)
                self.write_txt(class_name,merge_lexicon)
                print("merge",class_name,"/",len(merge_lexicon))
                counter = counter + len(merge_lexicon)

            else: 
                self.write_txt(class_name,class_lexicon)
                counter = counter + len(class_lexicon)
        print("number of term in Lexicon: ", counter)
        #print("self.export_class_name:", self.export_class_name)
        
        return self.export_class_name
    
    def write_txt(self,class_name,class_lexicon):
        """
        write lexicon txt file of input lexicon
        :Input 1: [String], class_name
        :Input 2: [Set], lexicon of each class
        :Output:  [.txt file]
        """
        lines = sorted(class_lexicon)
        
        flag = re.search('[A-Za-z-]+',class_name)
        class_name = class_name[flag.span()[0]:flag.span()[1]] #
        
        self.export_class_name.append(class_name.lower())
        
        with open('Lexicon_merge/{}.txt'.format(class_name.lower()), 'w', encoding='UTF-8') as f:
            for line in lines:
                if line:
                    #line = line.replace(" ", "")
                    f.write(line.strip('\n'))
                    f.write('\n')
                
    def read_pchome_file(self,file_path):
        """
        read external lexicon (txt) 
        :Input 1: [String], file_path
        """
        # read file
        pchome_file = []
        with open(file_path,encoding="utf-8") as f:
            lines = f.readlines()
            for i,line in enumerate(lines):
                line = re.sub('[^\u4e00-\u9fa5A-Za-z0-9]+','',line) #只留下中文、英文、數字
                pchome_file.append(line.strip('\n'))
        
        pchome_file = set(pchome_file)
        
        return pchome_file
    
    def merge_non_english(self,text):
        '''
        merge non english character together
        :Input: [String], e.g. "this is a 範 例 。"
        :Output: [String], e.g. "this is a 範例。"
        '''
        while re.search('[^A-Za-z\s]+\s[^A-Za-z\s]+',text):
            flag = re.search('[^A-Za-z\s]+\s[^A-Za-z\s]+',text)
            text = text[:flag.span()[0]] + text[flag.span()[0]:flag.span()[1]].replace(" ", "") + text[flag.span()[1]:]
            
        return text
        
    def class_lexicon(self,class_name):
        '''
        Return class lexicon that have no duplicate
        :Input: [String], class name of cilin
        :Output: [list], lexicon of corresponding class name [String, String, String, String, ... , String]
        '''
        class_lexicon = []
        #############################################
        ## 將 class_name 所對應到的詞林大類裡，所有 term 取出，放到 class_lexicon 中
        #############################################
        
        for _, (k_2, v_2) in enumerate(self.data[class_name].items()): #中類 type dict
            if class_name == 'Market 商用詞':
                if _ >= 1:  # 只針對 'Market 商用詞' 的第一個中類輸出字典
                    break
            for idx_3, v_3 in enumerate(v_2): #小類 type list

                for idx_4 ,g_w in enumerate(v_3):
                    if isinstance(g_w, list): #相似詞
                        for idx_5 ,s_w in enumerate(g_w):
                            if isinstance(s_w, list): #同義詞
                                for term in s_w:
                                    if '##' not in term:
                                        #term = self.merge_non_english(term) # merge non english character together
                                        #class_lexicon.append(term.lower()) #計算數量時關閉
                                        term = re.sub('[^\u4e00-\u9fa5A-Za-z0-9]+','',term) #只留下中文、英文、數字
                                        class_lexicon.append(term.lower())
                            else:
                                if '##' not in s_w:
                                    #term = self.merge_non_english(s_w) # merge non english character together
                                    #class_lexicon.append(s_w.lower()) #計算數量時關閉
                                    term = re.sub('[^\u4e00-\u9fa5A-Za-z0-9]+','',s_w) #只留下中文、英文、數字
                                    class_lexicon.append(term.lower())
                    else:
                        if '##' not in g_w:
                            #term = self.merge_non_english(g_w) # merge non english character together
                            #class_lexicon.append(g_w.lower()) #計算數量時關閉
                            term = re.sub('[^\u4e00-\u9fa5A-Za-z0-9]+','',g_w) #只留下中文、英文、數字
                            class_lexicon.append(term.lower())
        
        class_lexicon = set(class_lexicon) # delete duplicate in class_lexicon

        if '##更改' in class_lexicon: 
            class_lexicon.remove('##更改') # delete '##更改' in class_lexicon

        if '' in class_lexicon: 
            class_lexicon.remove('') # delete '' in class_lexicon

        return class_lexicon

In [3]:
if __name__ == '__main__':
    o = OutPutLexicon('cilin_labeled_data_.json') 
    o.start()

Brand 品牌 / 2284
merge Brand 品牌 / 3299
Name 名稱 / 1146
Model 型號 / 1041
Type 種類 / 11553
merge Type 種類 / 11575
Part 商品部位 / 130
Customer 客群 / 97
Event 事件 / 79
Location 地名 / 90
Place 場所 / 42
Ad 廣告詞 / 601
Market 商用詞 / 231
P-Size 大小 / 58
P-Material 材料 / 332
P-Shape 外型 / 243
P-Color 顏色 / 373
P-TP 紋理花紋 / 100
P-Flavor 味 / 86
P-Theme 主題 / 463
P-Other 其他特徵 / 1236
Time 時間 / 52
Quantity 量 / 26
Number 數 / 250
Operator 運算子 / 7
Unit 單位 / 194
test 測試 / 10
number of term in Lexicon:  24045


### sandbox

In [4]:
# term = " watch 膜"
# re.sub('[^\u4e00-\u9fa5A-Za-z0-9]+','',term)

'watch膜'

In [10]:
# a = [[1,2,3],[4,5,6],[99]]
# b = [[99]]
# s = set(tuple(i) for i in b)&set(tuple(i) for i in a)
# [list(t) for t in s]

[[99]]